In [13]:
# generate an nlp model to cluster 21 different appartments based on descriptions and slightly fluctauting prices

# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from unidecode import unidecode

In [14]:
# import data
jan_apr_desc = pd.read_csv('dados_afonso/room_say_report/reservas_por_data_criada/jan_apr.csv')
may_june_desc = pd.read_csv('dados_afonso/room_say_report/reservas_por_data_criada/may_june.csv')
jul_aug_desc = pd.read_csv('dados_afonso/room_say_report/reservas_por_data_criada/jul_aug.csv')

data_desc = pd.concat([jan_apr_desc, may_june_desc, jul_aug_desc])

jan_mar_pri = pd.read_csv('dados_afonso/reservation_summary/reservas_por_data_criada/jan_mar.csv')
apr_jun_pri = pd.read_csv('dados_afonso/reservation_summary/reservas_por_data_criada/apr_jun.csv')
jul_set_pri = pd.read_csv('dados_afonso/reservation_summary/reservas_por_data_criada/jul_set.csv')
aug_pri = pd.read_csv('dados_afonso/reservation_summary/reservas_por_data_criada/aug.csv')

data_pri = pd.concat([jan_mar_pri, apr_jun_pri, jul_set_pri, aug_pri])


In [15]:
data = pd.merge(data_desc, data_pri, on='Reservation ID', how='left')

data = data[data['Channel_x'] == 'Booking.com']

In [16]:
data_proc = data[['Reservation ID', 'Room', 'Total Amount']]

In [17]:
data_proc.rename(columns={'Reservation ID' : 'id', 'Room' : 'desc', 'Total Amount': 'price'}, inplace=True)

/tmp/ipykernel_1319/595610279.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_proc.rename(columns={'Reservation ID' : 'id', 'Room' : 'desc', 'Total Amount': 'price'}, inplace=True)


In [18]:
data.shape

(1766, 21)

In [19]:
data_proc.price = data_proc.price.str[:-5]

/tmp/ipykernel_1319/1310291950.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_proc.price = data_proc.price.str[:-5]


In [20]:
data_proc.price = data_proc.price.astype('float')

/tmp/ipykernel_1319/2401445117.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_proc.price = data_proc.price.astype('float')


In [53]:
# use nlp to cluster appartments based on descriptions
data_proc = data_proc.drop_duplicates(subset='id', keep=False)
data = data.drop_duplicates(subset='Reservation ID', keep=False)

def clean_text(text):
    text = unidecode(text)
    text = text.lower()
    return text

data_proc['mod_desc'] = data_proc['desc'].apply(clean_text)

# select number before "quarto" and make it a column
def add_n_rooms(text):
    n_rooms = re.findall(r'(\d+)\s+quarto', text)
    if len(n_rooms) > 0:
        return int(n_rooms[0])
    else:
        return 1
    
data_proc['n_rooms'] = data_proc['mod_desc'].apply(add_n_rooms)

# add a columns for road
def add_road(text):
    rua = re.findall(r'rua\s+(\w+)', text)
    if len(rua) > 0:
        return rua[0]
    else:
        return 'no_road'
    
data_proc['road'] = data_proc['mod_desc'].apply(add_road)

# add a column for terreo
def add_ground(text):
    ground = re.findall(r'terreo', text)
    if len(ground) > 0:
        return 1
    else:
        return 0
    
data_proc['ground'] = data_proc['mod_desc'].apply(add_ground)

# add a column for the door number
def add_door(text):
    text = text.replace('n.º', '')
    door = re.search(r"rua.+?,\s*(\d{1,2}\b)", text)
        
    if door is not None:
        return int(door.group(1))
    else:
        return 0
    
data_proc['door'] = data_proc['mod_desc'].apply(add_door)

# add a column for terraco
def add_terrace(text):
    terrace = re.findall(r'terraco', text)
    if len(terrace) > 0:
        return 1
    else:
        return 0
    
data_proc['terrace'] = data_proc['mod_desc'].apply(add_terrace)

# add a column for view of the river
def add_river(text):
    river = re.findall(r'rio', text)
    if len(river) > 0:
        return 1
    else:
        return 0
    
data_proc['river'] = data_proc['mod_desc'].apply(add_river)

# remove stuff that I've noticed is not relevant
def remove_manually(text):
    text = re.sub(r'\d{1,2}%', '', text)
    text = text.replace('n.º', '')
    text = text.replace('united states pos', '')
    text = text.replace('nao reembolsavel', '')
    text = text.replace('-', '')
    text = text.replace('general', '')
    text = text.replace('oferta', '')
    text = text.replace('apartamento', '')
    text = text.replace('com', '')
    text = text.replace('high season', '')
    text = text.replace('night', '')
    text = text.replace('reserva antecipada', '')
    text = re.sub(r'\d{1,2} de \w{3} de', '', text)
    text = re.sub(r'\w{3} \d{1,2}', '', text)
    text = re.sub(r'\d', '', text)
    return text


data_proc['mod_desc'] = data_proc['mod_desc'].apply(remove_manually)


# lemmatize data
lemmatizer = WordNetLemmatizer()
def lemmatize_text(text):
    text = [lemmatizer.lemmatize(word) for word in text.split()]
    return " ".join(text)

data_proc['mod_desc'] = data_proc['mod_desc'].apply(lemmatize_text)

# stem data
stemmer = PorterStemmer()
def stem_text(text):
    text = [stemmer.stem(word) for word in text.split()]
    return " ".join(text)

data_proc['mod_desc'] = data_proc['mod_desc'].apply(stem_text)

# remove stopwords
stop_words = set(stopwords.words('portuguese'))

def remove_stopwords(text):
    text = [word for word in text.split() if word not in stop_words]
    return " ".join(text)

data_proc['mod_desc'] = data_proc['mod_desc'].apply(remove_stopwords)

# vectorize data
vectorizer = TfidfVectorizer()

# one hot encode road
road_ohe = pd.get_dummies(data_proc['road'])

# define X, which contains a vector for each description and a price
vectors = vectorizer.fit_transform(data_proc['mod_desc'])
vectors_df = pd.DataFrame(vectors.toarray())
price = data_proc['price']
n_rooms = data_proc['n_rooms']
road_ohe = pd.get_dummies(data_proc['road'])
terrace = data_proc['terrace']
ground = data_proc['ground']
door = data_proc['door']
door_ohe = pd.get_dummies(data_proc['door'])
river = data_proc['river']
door_ohe = pd.get_dummies(data_proc['door'])
X = np.column_stack((vectors_df, n_rooms, road_ohe, terrace, ground, door_ohe, river))


# cluster data
kmeans_room = KMeans(n_clusters=21, random_state=0).fit(X)
kmeans_type = KMeans(n_clusters=14, random_state=0).fit(X)

# add cluster column to data
data_proc['cluster_room'] = kmeans_room.labels_
data_proc['cluster_type'] = kmeans_type.labels_

# add cluster column to data
data['cluster_room'] = kmeans_room.labels_
data['cluster_type'] = kmeans_type.labels_
data['n_rooms'] = n_rooms
data['door'] = door

In [36]:
data.door.value_counts()


0     1020
3      190
19      91
30      87
Name: door, dtype: int64

In [37]:
data_proc.groupby('cluster_type').cluster_room.nunique()

cluster_type
0    4
1    3
2    4
3    1
4    3
5    2
6    1
7    2
8    1
Name: cluster_room, dtype: int64

In [38]:
data_proc.groupby('cluster_type').n_rooms.nunique()

cluster_type
0    1
1    1
2    2
3    1
4    1
5    1
6    1
7    1
8    1
Name: n_rooms, dtype: int64

In [70]:
data.Room[data.cluster_type == 0].values

array(['Apartamento com 2 Quartos (4 Adultos) - Rua dos Remédios, n.º 30 - General - 23% - Oferta Básica - 3 de abr de 2022',
       'Apartamento com 2 Quartos (4 Adultos) - Rua dos Remédios, n.º 30 - General - 23% - Oferta Básica - 3 de abr de 2022',
       'Apartamento com 2 Quartos (4 Adultos) - Rua dos Remédios, n.º 30 - Não reembolsável - 23% - Oferta Básica - 3 de abr de 2022',
       'Apartamento com 2 Quartos (4 Adultos) - Rua dos Remédios, n.º 30 - General - 23% - Oferta Básica - 3 de abr de 2022',
       'Apartamento com 2 Quartos (4 Adultos) - Rua dos Remédios, n.º 30 - Não reembolsável - 23% - Oferta Básica - 3 de abr de 2022',
       'Apartamento com 2 Quartos (4 Adultos) - Rua dos Remédios, n.º 30 - General - 23% - Oferta Básica - 3 de abr de 2022',
       'Apartamento com 2 Quartos (4 Adultos) - Rua dos Remédios, n.º 30 - Não reembolsável - 23% - Oferta Básica - 3 de abr de 2022',
       'Apartamento com 2 Quartos (4 Adultos) - Rua dos Remédios, n.º 30 - General - 23% - 

In [26]:
data.loc[92]['Room']

'Apartamento com 1 Quarto - Rua de Santo Estêvão, 19'

In [54]:
add_door('Apartamento com 1 Quarto (2 Adultos) - Rua dos Remédios, 30'.lower())

30